In [1]:
import pandas as pd
import numpy as np
import datetime
import os
from scipy.stats import fisher_exact
from statsmodels.stats.multitest import multipletests
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import ipywidgets as widgets
from ipywidgets import interact, interactive

In [2]:
# output directory
direction = "down"
species = "MR"
now = datetime.datetime.now()
line_enrichment = f'../data/line_enrichment_{direction}_{species}_{now.strftime("%y%m")}'
os.makedirs(line_enrichment, exist_ok=True)

In [3]:
def count_goslim(df, file_name: str):
    unique_gene_pairs = df.drop_duplicates(subset=['GOSlim GOA Accession(s)', 'Gene stable ID_mouse', 'Gene stable ID_rice'])
    goslim_counts = unique_gene_pairs.groupby(['GOSlim GOA Accession(s)', 'GOSlim GOA Description', 'GOSlim_domain']).size().reset_index(name='counts')
    goslim_counts = goslim_counts.sort_values(by='counts', ascending=False).copy()
    goslim_counts.reset_index(drop=True, inplace=True)
    results_directory = f'../data/{line_enrichment}/goslim_correspondence_counts_{direction}_{species}'
    os.makedirs(results_directory, exist_ok=True)
    goslim_counts.to_csv(f'{results_directory}/{file_name}', sep='\t', index=False)
    return goslim_counts

In [5]:
merged_df_goslim_pd = pd.read_csv('../data/GOSlim_merge_MR/GOslim_merge_common_goslim_correspondence_all.tsv', 
                                  sep='\t',
                                  low_memory=False)
merged_df_up = pd.read_csv(f'../data/circos_{direction}_{species}_2312/combined_goslim_down.tsv', sep='\t')

display(merged_df_goslim_pd, merged_df_up)

,GOSlim GOA Accession(s),GOSlim GOA Description,GOSlim_domain,Gene stable ID_mouse,Gene stable ID_rice
0,GO:0003723,RNA binding,molecular_function,ENSMUSG00000064336,Os01g0100700
1,GO:0003723,RNA binding,molecular_function,ENSMUSG00000064336,Os01g0101150
2,GO:0003723,RNA binding,molecular_function,ENSMUSG00000064336,Os01g0101300
3,GO:0003723,RNA binding,molecular_function,ENSMUSG00000064336,Os01g0101600
4,GO:0003723,RNA binding,molecular_function,ENSMUSG00000064336,Os01g0118100
...,...,...,...,...,...
183104384,GO:0030312,external encapsulating structure,cellular_component,ENSMUSG00000026938,Os12g0514500
183104385,GO:0030312,external encapsulating structure,cellular_component,ENSMUSG00000026938,Os12g0530100
183104386,GO:0030312,external encapsulating structure,cellular_component,ENSMUSG00000026938,Os12g0546800
183104387,GO:0030312,external encapsulating structure,cellular_component,ENSMUSG00000026938,Os12g0563700


,GOSlim GOA Accession(s),GOSlim GOA Description,GOSlim_domain,Gene stable ID_rice,Gene stable ID_mouse,Chromosome/scaffold name_rice,start1 (bp),end1 (bp),Chromosome/scaffold name_mouse,start2 (bp),end2 (bp),HN-score(HN5)_rice,HN-score(HN5)_mouse
0,GO:0003824,catalytic activity,molecular_function,Os03g0358800,ENSMUSG00000039936,R_3,13915940,13917546,M_4,149733625,149787028,-40,-15
1,GO:0003824,catalytic activity,molecular_function,Os10g0527601,ENSMUSG00000071724,R_10,20493427,20494200,M_15,76178548,76181096,-55,-17
2,GO:0003824,catalytic activity,molecular_function,Os04g0493400,ENSMUSG00000071724,R_4,24687753,24689297,M_15,76178548,76181096,-61,-17
3,GO:0003824,catalytic activity,molecular_function,Os01g0591300,ENSMUSG00000071724,R_1,23113647,23120516,M_15,76178548,76181096,-60,-17
4,GO:0003824,catalytic activity,molecular_function,Os02g0240300,ENSMUSG00000071724,R_2,7948543,7949997,M_15,76178548,76181096,-60,-17
...,...,...,...,...,...,...,...,...,...,...,...,...,...
16810,GO:0005764,lysosome,cellular_component,Os02g0715000,ENSMUSG00000020323,R_2,29667743,29669572,M_10,79617308,79626795,-41,-17
16811,GO:0005764,lysosome,cellular_component,Os01g0613500,ENSMUSG00000020323,R_1,24343518,24345172,M_10,79617308,79626795,-75,-17
16812,GO:0005764,lysosome,cellular_component,Os01g0971400,ENSMUSG00000020323,R_1,42855657,42857462,M_10,79617308,79626795,-41,-17
16813,GO:0005635,nuclear envelope,cellular_component,Os12g0236100,ENSMUSG00000037997,R_12,7441389,7446142,M_6,127423803,127471224,-81,-16


In [6]:
gosilm_correspondence_up_counts = count_goslim(merged_df_up, f'goslim_correspondence_counts_{direction}.tsv')
goslim_correspondence_all_counts = count_goslim(merged_df_goslim_pd, 'goslim_correspondence_counts_all.tsv')

display(gosilm_correspondence_up_counts, goslim_correspondence_all_counts)

,GOSlim GOA Accession(s),GOSlim GOA Description,GOSlim_domain,counts
0,GO:0003824,catalytic activity,molecular_function,7482
1,GO:0005634,nucleus,cellular_component,2520
2,GO:0005886,plasma membrane,cellular_component,1110
3,GO:0016787,hydrolase activity,molecular_function,984
4,GO:0005576,extracellular region,cellular_component,841
5,GO:0016740,transferase activity,molecular_function,840
6,GO:0003677,DNA binding,molecular_function,736
7,GO:0048856,anatomical structure development,biological_process,530
8,GO:0036211,protein modification process,biological_process,450
9,GO:0005215,transporter activity,molecular_function,308


,GOSlim GOA Accession(s),GOSlim GOA Description,GOSlim_domain,counts
0,GO:0003824,catalytic activity,molecular_function,56414781
1,GO:0005634,nucleus,cellular_component,46028115
2,GO:0005886,plasma membrane,cellular_component,13465191
3,GO:0036211,protein modification process,biological_process,10391837
4,GO:0016740,transferase activity,molecular_function,9615454
5,GO:0016787,hydrolase activity,molecular_function,7777545
6,GO:0048856,anatomical structure development,biological_process,7211128
7,GO:0005829,cytosol,cellular_component,5489678
8,GO:0003677,DNA binding,molecular_function,4990802
9,GO:0003723,RNA binding,molecular_function,3071712


In [7]:
# for debugging purpose
total_counts_up = gosilm_correspondence_up_counts['counts'].sum()
total_counts_all = goslim_correspondence_all_counts['counts'].sum()

unique_rows_up = merged_df_up.drop_duplicates(subset=['GOSlim GOA Accession(s)', 'Gene stable ID_mouse', 'Gene stable ID_rice']).shape[0]
unique_rows_all = merged_df_goslim_pd.drop_duplicates(subset=['GOSlim GOA Accession(s)', 'Gene stable ID_mouse', 'Gene stable ID_rice']).shape[0]

print(f"Total counts: {unique_rows_all}")
print(f"up counts: {unique_rows_up}")
print(f"Counts match rows: {unique_rows_up == total_counts_up}")

Total counts: 183104389
up counts: 16815
Counts match rows: True


In [8]:
# fold enrichment
def fold_enrichment(df_goslim_up_counts, df_goslim_all_counts, unique_genes_up, unique_genes_all, file_name: str):
    """_summary_
    Args:
        df_goslim_up_counts (dataframe): _description_
        df_goslim_all_counts (dataframe): _description_
        unique_genes_up (): _description_
        unique_genes_all (dataframe): _description_
        file_name (str): _description_

    Returns:
        dataframe : _description_
    """
    merged_df = pd.merge(df_goslim_up_counts, 
                         df_goslim_all_counts,
                         on=['GOSlim GOA Accession(s)', 'GOSlim GOA Description', 'GOSlim_domain'], 
                         how='right', # if there is no match, fill with NaN or 0
                         suffixes=('_down', '_all'))
    
    merged_df['counts_down'] = merged_df['counts_down'].fillna(0)
    merged_df['counts_all'] = merged_df['counts_all'].fillna(0)
    merged_df['down_ratio'] = merged_df['counts_down'] / unique_genes_up
    merged_df['all_ratio'] = merged_df['counts_all'] / unique_genes_all
    merged_df['fold_enrichment'] = merged_df['down_ratio'] / merged_df['all_ratio']
    merged_df.replace([np.inf, -np.inf], np.nan, inplace=True) # inf -> nan
    results_directory = f'../data/{line_enrichment}/goslim_fold_enrichment_correspondence_{direction}_{species}'
    os.makedirs(results_directory, exist_ok=True)
    merged_df.to_csv(f'{results_directory}/{file_name}', sep='\t', index=False)
    
    return merged_df


goslim_correspondence_enrichment = fold_enrichment(gosilm_correspondence_up_counts, 
                                                   goslim_correspondence_all_counts, 
                                                   unique_rows_up, 
                                                   unique_rows_all, 
                                                   f'goslim_correspondence_fold_enrichment_{direction}.tsv')
display(goslim_correspondence_enrichment)

,GOSlim GOA Accession(s),GOSlim GOA Description,GOSlim_domain,counts_down,counts_all,down_ratio,all_ratio,fold_enrichment
0,GO:0003824,catalytic activity,molecular_function,7482.0,56414781,0.444960,0.308102,1.444198
1,GO:0005634,nucleus,cellular_component,2520.0,46028115,0.149866,0.251376,0.596183
2,GO:0005886,plasma membrane,cellular_component,1110.0,13465191,0.066012,0.073538,0.897661
3,GO:0036211,protein modification process,biological_process,450.0,10391837,0.026762,0.056754,0.471544
4,GO:0016740,transferase activity,molecular_function,840.0,9615454,0.049955,0.052514,0.951287
5,GO:0016787,hydrolase activity,molecular_function,984.0,7777545,0.058519,0.042476,1.377699
6,GO:0048856,anatomical structure development,biological_process,530.0,7211128,0.031519,0.039383,0.800340
7,GO:0005829,cytosol,cellular_component,261.0,5489678,0.015522,0.029981,0.517721
8,GO:0003677,DNA binding,molecular_function,736.0,4990802,0.043770,0.027257,1.605866
9,GO:0003723,RNA binding,molecular_function,0.0,3071712,0.000000,0.016776,0.000000


## Q-value

In [9]:
def calculate_p_q_values(df_enrichment, unique_correspondence_down, unique_correspondence_all, file_name: str):
    p_values = []
    df_enrichment[f'counts_{direction}'] = df_enrichment[f'counts_{direction}'].astype(int) # float(0.0) to int(0)
    for index, row in df_enrichment.iterrows():
        if row[f'counts_{direction}'] == 0:
            p_values.append(1.0)
        else:
            observed_down_correspondence = row[f'counts_{direction}'] # create contingency table
            observed_all_correspondence = row['counts_all']
            observed_not_down_correspondence = observed_all_correspondence - observed_down_correspondence
            total_not_down_correspondence = unique_correspondence_all - unique_correspondence_down
            table = [
            [observed_down_correspondence, unique_correspondence_down - observed_down_correspondence],
            [observed_not_down_correspondence, total_not_down_correspondence - observed_not_down_correspondence]
            ]
            _, p_value = fisher_exact(table, alternative='greater') # Fisher's exact test
            p_values.append(p_value)
    
    _, q_values, _, _ = multipletests(p_values, 
                                      alpha=0.05, 
                                      method='fdr_bh') # Benjamini/Hochberg method
    
    df_enrichment['p_value'] = p_values
    df_enrichment['q_value'] = q_values

    results_directory = f'../data/{line_enrichment}/goslim_correspondence_q_values_{direction}_{species}'
    os.makedirs(results_directory, exist_ok=True)
    df_enrichment['GOSlim'] = df_enrichment['GOSlim GOA Accession(s)'] + ": " + df_enrichment['GOSlim GOA Description']
    cols = df_enrichment.columns.tolist()
    cols = [cols[-1]] + cols[:-1]
    df_enrichment = df_enrichment[cols]
    df_enrichment.drop(['GOSlim GOA Accession(s)', 'GOSlim GOA Description'], axis=1)
    df_enrichment = df_enrichment[df_enrichment[f'counts_{direction}'] != 0]
    df_enrichment = df_enrichment.sort_values(by='fold_enrichment', ascending=False)
    df_enrichment.reset_index(drop=True, inplace=True)
    df_enrichment.to_csv(f'{results_directory}/{file_name}', sep='\t', index=False)

    return df_enrichment

goslim_correspondence_with_p_q = calculate_p_q_values(goslim_correspondence_enrichment.copy(),
                                                      unique_rows_up,
                                                      unique_rows_all, 
                                                      f'goslim_correspondence_fold_enrichment_p_q_{direction}.tsv')


display(goslim_correspondence_with_p_q)

,GOSlim,GOSlim GOA Accession(s),GOSlim GOA Description,GOSlim_domain,counts_down,counts_all,down_ratio,all_ratio,fold_enrichment,p_value,q_value
0,GO:0005576: extracellular region,GO:0005576,extracellular region,cellular_component,841,2069023,0.050015,0.011300,4.426216,1.028298e-268,1.748106e-267
1,GO:0005764: lysosome,GO:0005764,lysosome,cellular_component,6,27234,0.000357,0.000149,2.399063,4.206618e-02,1.589167e-01
2,GO:0008289: lipid binding,GO:0008289,lipid binding,molecular_function,56,257535,0.003330,0.001406,2.367847,1.063385e-08,6.025850e-08
3,GO:0005215: transporter activity,GO:0005215,transporter activity,molecular_function,308,1687420,0.018317,0.009216,1.987602,8.451898e-28,7.184113e-27
4,GO:0005975: carbohydrate metabolic process,GO:0005975,carbohydrate metabolic process,biological_process,102,655182,0.006066,0.003578,1.695275,5.390000e-07,2.618000e-06
5,GO:0003677: DNA binding,GO:0003677,DNA binding,molecular_function,736,4990802,0.043770,0.027257,1.605866,5.572734e-34,6.315766e-33
6,GO:0003824: catalytic activity,GO:0003824,catalytic activity,molecular_function,7482,56414781,0.444960,0.308102,1.444198,1.324139e-303,4.502072e-302
7,GO:0016787: hydrolase activity,GO:0016787,hydrolase activity,molecular_function,984,7777545,0.058519,0.042476,1.377699,7.895711e-23,5.369083e-22
8,GO:0006091: generation of precursor metabolite...,GO:0006091,generation of precursor metabolites and energy,biological_process,27,243045,0.001606,0.001327,1.209704,1.856097e-01,6.310729e-01
9,GO:0006629: lipid metabolic process,GO:0006629,lipid metabolic process,biological_process,153,1429560,0.009099,0.007807,1.165443,3.387015e-02,1.439482e-01


In [10]:
#data preprocessing for plotting
goslim_correspondence_with_p_q.rename(columns={f'counts_{direction}': 'line counts'}, inplace=True)

if 'p_value' in goslim_correspondence_with_p_q.columns:
    goslim_correspondence_with_p_q['-log10(p-value)'] = -np.log10(goslim_correspondence_with_p_q['p_value'])
    
if 'q_value' in goslim_correspondence_with_p_q.columns:
    goslim_correspondence_with_p_q['-log10(q-value)'] = -np.log10(goslim_correspondence_with_p_q['q_value'])


cc = goslim_correspondence_with_p_q.loc[goslim_correspondence_with_p_q['GOSlim_domain'] == 'cellular_component']
bp = goslim_correspondence_with_p_q.loc[goslim_correspondence_with_p_q['GOSlim_domain'] == 'biological_process']
mf = goslim_correspondence_with_p_q.loc[goslim_correspondence_with_p_q['GOSlim_domain'] == 'molecular_function']

cc.reset_index(drop=True, inplace=True)
bp.reset_index(drop=True, inplace=True)
mf.reset_index(drop=True, inplace=True)

display(cc, bp, mf)

,GOSlim,GOSlim GOA Accession(s),GOSlim GOA Description,GOSlim_domain,line counts,counts_all,down_ratio,all_ratio,fold_enrichment,p_value,q_value,-log10(p-value),-log10(q-value)
0,GO:0005576: extracellular region,GO:0005576,extracellular region,cellular_component,841,2069023,0.050015,0.011300,4.426216,1.028298e-268,1.748106e-267,267.987881,266.757432
1,GO:0005764: lysosome,GO:0005764,lysosome,cellular_component,6,27234,0.000357,0.000149,2.399063,4.206618e-02,1.589167e-01,1.376067,0.798831
2,GO:0005886: plasma membrane,GO:0005886,plasma membrane,cellular_component,1110,13465191,0.066012,0.073538,0.897661,9.999309e-01,1.000000e+00,0.000030,-0.000000
3,GO:0030312: external encapsulating structure,GO:0030312,external encapsulating structure,cellular_component,12,174151,0.000714,0.000951,0.750338,8.727785e-01,1.000000e+00,0.059096,-0.000000
4,GO:0005773: vacuole,GO:0005773,vacuole,cellular_component,18,270510,0.001070,0.001477,0.724588,9.359822e-01,1.000000e+00,0.028732,-0.000000
5,GO:0005634: nucleus,GO:0005634,nucleus,cellular_component,2520,46028115,0.149866,0.251376,0.596183,1.000000e+00,1.000000e+00,-0.000000,-0.000000
6,GO:0005856: cytoskeleton,GO:0005856,cytoskeleton,cellular_component,32,661920,0.001903,0.003615,0.526437,9.999815e-01,1.000000e+00,0.000008,-0.000000
7,GO:0005829: cytosol,GO:0005829,cytosol,cellular_component,261,5489678,0.015522,0.029981,0.517721,1.000000e+00,1.000000e+00,-0.000000,-0.000000
8,GO:0005635: nuclear envelope,GO:0005635,nuclear envelope,cellular_component,2,45942,0.000119,0.000251,0.474048,9.232444e-01,1.000000e+00,0.034683,-0.000000
9,GO:0005739: mitochondrion,GO:0005739,mitochondrion,cellular_component,31,2075124,0.001844,0.011333,0.162675,1.000000e+00,1.000000e+00,-0.000000,-0.000000


,GOSlim,GOSlim GOA Accession(s),GOSlim GOA Description,GOSlim_domain,line counts,counts_all,down_ratio,all_ratio,fold_enrichment,p_value,q_value,-log10(p-value),-log10(q-value)
0,GO:0005975: carbohydrate metabolic process,GO:0005975,carbohydrate metabolic process,biological_process,102,655182,0.006066,0.003578,1.695275,5.390000e-07,0.000003,6.268411e+00,5.582030
1,GO:0006091: generation of precursor metabolite...,GO:0006091,generation of precursor metabolites and energy,biological_process,27,243045,0.001606,0.001327,1.209704,1.856097e-01,0.631073,7.313994e-01,0.199920
2,GO:0006629: lipid metabolic process,GO:0006629,lipid metabolic process,biological_process,153,1429560,0.009099,0.007807,1.165443,3.387015e-02,0.143948,1.470183e+00,0.841794
3,GO:0048856: anatomical structure development,GO:0048856,anatomical structure development,biological_process,530,7211128,0.031519,0.039383,0.800340,1.000000e+00,1.000000,1.153370e-08,-0.000000
4,GO:0030154: cell differentiation,GO:0030154,cell differentiation,biological_process,84,1302168,0.004996,0.007112,0.702448,9.997548e-01,1.000000,1.064883e-04,-0.000000
5,GO:0036211: protein modification process,GO:0036211,protein modification process,biological_process,450,10391837,0.026762,0.056754,0.471544,1.000000e+00,1.000000,-0.000000e+00,-0.000000


,GOSlim,GOSlim GOA Accession(s),GOSlim GOA Description,GOSlim_domain,line counts,counts_all,down_ratio,all_ratio,fold_enrichment,p_value,q_value,-log10(p-value),-log10(q-value)
0,GO:0008289: lipid binding,GO:0008289,lipid binding,molecular_function,56,257535,0.003330,0.001406,2.367847,1.063385e-08,6.025850e-08,7.973309,7.219982
1,GO:0005215: transporter activity,GO:0005215,transporter activity,molecular_function,308,1687420,0.018317,0.009216,1.987602,8.451898e-28,7.184113e-27,27.073046,26.143627
2,GO:0003677: DNA binding,GO:0003677,DNA binding,molecular_function,736,4990802,0.043770,0.027257,1.605866,5.572734e-34,6.315766e-33,33.253932,32.199574
3,GO:0003824: catalytic activity,GO:0003824,catalytic activity,molecular_function,7482,56414781,0.444960,0.308102,1.444198,1.324139e-303,4.502072e-302,302.878067,301.346588
4,GO:0016787: hydrolase activity,GO:0016787,hydrolase activity,molecular_function,984,7777545,0.058519,0.042476,1.377699,7.895711e-23,5.369083e-22,22.102609,21.270100
5,GO:0140110: transcription regulator activity,GO:0140110,transcription regulator activity,molecular_function,190,1948470,0.011299,0.010641,1.061847,2.122061e-01,6.559098e-01,0.673242,0.183156
6,GO:0016740: transferase activity,GO:0016740,transferase activity,molecular_function,840,9615454,0.049955,0.052514,0.951287,9.346300e-01,1.000000e+00,0.029360,-0.000000
7,GO:0005198: structural molecule activity,GO:0005198,structural molecule activity,molecular_function,20,488130,0.001189,0.002666,0.446166,9.999889e-01,1.000000e+00,0.000005,-0.000000


In [11]:
# enrichment result plot function
current_fig = None # global variable

def plot_goslim_dotplot_with_lines(data, col1, goslim_col, col2, col3, ylabel_suffix='', width=6, height=9, palette="flare"):

    # 1. figure size
    global current_fig
    figsize = (width, height)
    current_fig, ax = plt.subplots(figsize=figsize)

    # 2. color normalization
    norm = mcolors.Normalize(vmin=data[col3].min(), vmax=data[col3].max())

    # 3.color palette
    color_palette = sns.color_palette(palette, as_cmap=True)

    # 4.plotting
    sns.scatterplot(
        data=data,
        x=col1,
        y=goslim_col,
        size=col2, 
        hue=col3,     
        palette=palette, 
        legend='brief',
        ax=ax
    )
    # 5.lines 
    for index, row in data.iterrows():
        ax.plot([0, row[col1]], [row[goslim_col], row[goslim_col]], color=color_palette(norm(row[col3])), lw=1)
    
    ax.grid(color='b', linestyle=':', linewidth=0.1)
    ax.set_xlabel(col1)
    ylabel_text = 'Common GOSlim Terms' + ('' + ylabel_suffix  if ylabel_suffix else '' )
    ax.set_ylabel(ylabel_text)

    plt.show()

# execute only this function to plot
# plot_goslim_dotplot_with_lines(goslim_correspondence_with_p_q.copy(), 'fold_enrichment', 'GOSlim', 'line counts', 'q_value', ylabel_suffix='') # add copy() to avoid SettingWithCopyWarning
# plot_goslim_dotplot_with_lines(cc.copy(), 'fold_enrichment', 'GOSlim', 'line counts', 'q_value', ylabel_suffix=' (Cellular Component)')
# plot_goslim_dotplot_with_lines(bp.copy(), 'fold_enrichment', 'GOSlim', 'line counts', 'q_value', ylabel_suffix=' (Biological Process)')
# plot_goslim_dotplot_with_lines(mf.copy(), 'fold_enrichment', 'GOSlim', 'line counts', 'q_value', ylabel_suffix=' (Molecular Function)')

In [17]:
# interactive setting
filename_text = widgets.Text(value='', description='File Name:')
png_directory = f'../data/{line_enrichment}/png'
os.makedirs(png_directory, exist_ok=True)

def save_plot(button):
    global current_fig
    if current_fig:
        filename = filename_text.value
        if filename:  
            current_fig.savefig(f'{png_directory}/{filename}.png', dpi=800, bbox_inches='tight')
        else:
            current_fig.savefig(f'{png_directory}/enrichment_result.png', dpi=800, bbox_inches='tight')


save_button = widgets.Button(description="Save Plot")
save_button.on_click(save_plot)

def interactive_plot(data):
    w = interactive(
        plot_goslim_dotplot_with_lines,
        data=widgets.fixed(data),
        col1=widgets.Dropdown(options= [('fold enrichment', 'fold_enrichment'), 
                                        ('-log10(p-value)', '-log10(p-value)'), 
                                        ('-log10(q-value)', '-log10(q-value)')], 
                                        value='fold_enrichment', 
                                        description='x-axis:'),
        goslim_col=widgets.fixed('GOSlim'),
        col2=widgets.Dropdown(options=data.columns.tolist(), value='line counts', description='size:'),
        col3=widgets.Dropdown(options= [('fold enrichment', 'fold_enrichment'), 
                                        ('-log10(p-value)', '-log10(p-value)'), 
                                        ('-log10(q-value)', '-log10(q-value)')], 
                                        value='-log10(q-value)', 
                                        description='hue:'),
        ylabel_suffix=widgets.Dropdown(
            options=[('', ''), 
                     ('cc', '(Cellular Component)'), 
                     ('bp', '(Biological Process)'), 
                     ('mf', '(Molecular Function)')],
            value='',
            description='y-label suffix:'),
        width=widgets.IntSlider(min=1, max=30, step=1, value=6, description='width:', readout_format='0.0f'),
        height=widgets.IntSlider(min=1, max=30, step=1, value=9, description='height:', readout_format='0.0f'),
        palette=widgets.Dropdown(options=['flare', 'magma', 'viridis'], value='flare', description='color palette:')
        
    )
    display(w, filename_text, save_button)

In [18]:
interactive_plot(goslim_correspondence_with_p_q.copy())

interactive(children=(Dropdown(description='x-axis:', options=(('fold enrichment', 'fold_enrichment'), ('-log1…

Text(value='', description='File Name:')

Button(description='Save Plot', style=ButtonStyle())

In [19]:
interactive_plot(cc.copy())

interactive(children=(Dropdown(description='x-axis:', options=(('fold enrichment', 'fold_enrichment'), ('-log1…

Text(value='', description='File Name:')

Button(description='Save Plot', style=ButtonStyle())

In [20]:
interactive_plot(bp.copy())

interactive(children=(Dropdown(description='x-axis:', options=(('fold enrichment', 'fold_enrichment'), ('-log1…

Text(value='', description='File Name:')

Button(description='Save Plot', style=ButtonStyle())

In [16]:
interactive_plot(mf.copy())

interactive(children=(Dropdown(description='x-axis:', options=(('fold enrichment', 'fold_enrichment'), ('-log1…

Text(value='enrichment_result_down_bp', description='File Name:')

Button(description='Save Plot', style=ButtonStyle())